# Infrastructure Setup

In [ ]:
import os
from chi import context

# We select which chameleon site(either UC or TACC) we are using and choose the project
context.choose_site()
context.choose_project()
username = os.environ.get('USER').replace("_","-")

## Creating a 1-day lease

In [ ]:
from chi import lease
from datetime import timedelta

In [ ]:
node_type="compute_cascadelake_r"
instance_count = 2

In [ ]:
mpi_lease = lease.Lease(f"{username}-mpi-lease", duration=timedelta(days=1))

mpi_lease.add_node_reservation(amount=instance_count, node_type=node_type)
mpi_lease.add_fip_reservation(1)
mpi_lease.submit(idempotent=True)

## Creating server instances

In [ ]:
from chi import server

In [ ]:
mpi_master=server.Server(
        f"{username}-mpi-master",
        reservation_id=mpi_lease.node_reservations[0]["id"],
        image_name="Ubuntu22.04-HPC-MPI-Spack",
        network_name="sharednet1"
)
mpi_master.submit(idempotent=True)

mpi_workers = []
for i in range(instance_count - 1):
    mpi_worker=server.Server(
            f"{username}-mpi-worker-{i+1}",
            reservation_id=mpi_lease.node_reservations[0]["id"],
            image_name="Ubuntu22.04-HPC-MPI-Spack",
            network_name="sharednet1"
    )
    mpi_worker.submit(idempotent=True)
    mpi_workers.append(mpi_worker)

In [ ]:
fip = mpi_lease.get_reserved_floating_ips()[0]
mpi_master.associate_floating_ip(fip)

## Create inventory.ini to work with ansible

In [ ]:
with open("./inventory.ini", "w") as f:
    f.write("[master_node]\n")
    f.write(f"{mpi_master.name} ansible_host={fip}\n\n")
    
    f.write("[worker_nodes]\n")
    f.write("\n".join(f"{w.name} ansible_host={w.addresses['sharednet1'][0]['addr']}" for w in mpi_workers))
    f.write("\n\n")
    f.write("[worker_nodes:vars]\n")
    f.write(f"ansible_ssh_common_args='-o ProxyJump=cc@{fip}'")

## Use Ansible to create an MPI Cluster

In [ ]:
import ansible_runner
import tempfile
tmpdir = tempfile.TemporaryDirectory()
ansible_run = ansible_runner.run(
    private_data_dir=tmpdir.name,
    inventory=os.path.abspath("inventory.ini"),
    envvars = {
        "ANSIBLE_PYTHON_INTERPRETER": "/usr/bin/python3",
        "ANSIBLE_SSH_ARGS": f"-F {os.path.abspath("config")}",
    },
    extravars={
        "spack_packages": ["pdsh", "gromacs +mpi ^openmpi"]
    },
    playbook=os.path.abspath("mpi_cluster.yml"),
    verbosity=0
)

In [ ]:
mpi_master.upload("./examples/src/hello.c", "/home/cc/hello.c")
mpi_master.upload("./examples/mpi_jobs/run_hello.sh", "/home/cc/run_hello.sh")

In [ ]:
mpi_master.execute(f'bash -lc "source /home/cc/run_hello.sh {mpi_master_hostname} {",".join(mpi_worker_hostnames)}"')

## Gromacs Test

### Water_GMX50_bare

It is a prebuilt benchmark test provided by GROMACS. It contains a simulation of bulk water molecules using the TIP3P model

In [ ]:
mpi_master.upload("./examples/mpi_jobs/run_gromacs_water.sh", "/home/cc/run_gromacs_water.sh")

In [ ]:
mpi_master.execute(f'bash -lc "source /home/cc/run_gromacs_water.sh {mpi_master_hostname} {",".join(mpi_worker_hostnames)}"')